In [79]:
import pandas as pd
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from sklearn.decomposition import TruncatedSVD

In [80]:
stop_words = set(stopwords.words("english"))

PATTERN_S = re.compile("\'s")  # matches `'s` from text  
PATTERN_RN = re.compile("\\r\\n\\b") #matches `\r` and `\n`
PATTERN_PUNC = re.compile(r"[^\w\s]") # matches all non 0-9 A-z whitespace 


def clean_text(text):
    """
    Series of cleaning. String to lower case, remove non words characters and numbers (punctuation, curly brackets etc).
        text (str): input text
    return (str): modified initial text
    """
    text = text.lower()  # lowercase text
    # replace the matched string with ' '
    text = re.sub(PATTERN_S, ' ', text)
    text = re.sub(PATTERN_RN, ' ', text)
    text = re.sub(PATTERN_PUNC, ' ', text)
    return text

def tokenizer(description, stop_words, normalization):
    
    if normalization == 'lemmatize':
        # tokenize and lemmatize text
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(w) for w in word_tokenize(description)]
        
    elif normalization == 'stem':
        # tokenize and stem text
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(w) for w in word_tokenize(description)]
    
   # remove tokens length of 2 or below and make all lowercase and remove stop words
    tokens = [w.lower() for w in tokens if (w.lower() not in stop_words) and (len(w) > 2) and(w.isalpha())]
    
    return tokens    
    
def process_query(query, normalization):
    stop_words = set(stopwords.words("english"))
    
    return tokenizer(clean_text(query), stop_words, normalization)

In [81]:
# code example taken from https://towardsdatascience.com/build-a-text-recommendation-system-with-python-e8b95d9f251c
def retrieve_top_n(m, max_docs):
    # return the sum on all tokens of cosinus for each sentence
    if len(m.shape) > 1:
        cos_sim = np.mean(m, axis=0) 
    else: 
        cos_sim = m
    index = np.argsort(cos_sim)[::-1] # from highest idx to smallest score \
    mask = np.ones(len(cos_sim))
    mask = np.logical_or(cos_sim[index] != 0, mask) #eliminate 0 cosine distance
    best_index = index[mask][:max_docs]  
    return best_index

In [82]:
class LsiTfidfRecommenderSystem:
    def __init__(self, docs, num_concepts=100, alpha=1.0, beta=0.75, gamma=0.15):
        self.alpha, self.beta, self.gamma = alpha, beta, gamma
        
        # create a doc-term matrix out of our doc collection
        self.vec = TfidfVectorizer()
        doc_term_mat = self.vec.fit_transform([" ".join(docs[doc_id]) for doc_id in docs])
        self.svd = TruncatedSVD(n_components=num_concepts, random_state=42)
        result = self.svd.fit_transform(doc_term_mat)
        
        self.q_vecs = {}
        
        self.doc_vecs = result # document vectors in a matrix
        
    def retrieve_docs(self, query, max_docs=10, normalization='lemmatize'):
        query = ' '.join(process_query(query, normalization))
        
        if query not in self.q_vecs:
            q_vec = self.vec.transform([query])
            lsi_transform = self.svd.transform(q_vec)
            self.q_vecs[query] = lsi_transform
        
        ret_docs = {}
        
        mat = cosine_similarity(self.q_vecs[query], self.doc_vecs)
        best_index = retrieve_top_n(mat, max_docs=max_docs)
        
        return best_index

In [83]:
df = pd.read_pickle(r'assets/processed_df.pkl')
docs = dict(zip(df['naics'], df['lemmatized']))

In [84]:
lsi_model = LsiTfidfRecommenderSystem(docs)

In [85]:
best_index = lsi_model.retrieve_docs('Home improvement store', normalization='lemmatize')

In [86]:
naics_titles = pd.read_excel('assets/6-digit_2017_Codes.xlsx')
naics_titles['naics'] = naics_titles['naics'].astype(str)

In [87]:
df = df.merge(naics_titles, on='naics', how='outer')

In [88]:
df.iloc[best_index][['naics', 'title']]

,naics,title
578,442299,All Other Home Furnishings Stores
577,442291,Window Treatment Stores
576,442210,Floor Covering Stores
629,453998,All Other Miscellaneous Store Retailers (excep...
628,453991,Tobacco Stores
627,453930,Manufactured (Mobile) Home Dealers
625,453910,Pet and Pet Supplies Stores
626,453920,Art Dealers
624,453310,Used Merchandise Stores
623,453220,"Gift, Novelty, and Souvenir Stores"


In [89]:
relevant_naics = pd.read_pickle('assets/relevant_naics_df.pkl')

In [90]:
relevant_naics.head()

,query,relevant_naics
0,Home improvement store,"[444110, 444120, 444130, 444190, 444210, 44422..."
1,Diesel fuel supplier,"[424710, 424720, 424110, 424120, 424130, 42421..."
2,Church,"[813110, 813211, 813212, 813219, 813311, 81331..."
3,Farm,"[115116, 115111, 115112, 115113, 115114, 11511..."
4,Seed supplier,"[424910, 424920, 424930, 424940, 424950, 42499..."


In [91]:
max_docs = 10

In [92]:
def calc_pre_rec_at_n(ret_docs, reljudges, n=-1):
    """
    Calculate precision and recall at n for each query in ret_docs
    """
    
    pre_at_n, rec_at_n = {}, {}
    
    for k, v in ret_docs.items():
        if n > -1 and n <= len(ret_docs):
            s1 = set(v[:n])
        else:
            s1 = set(v)
        s2 = reljudges[k]
        precision = len(s1.intersection(s2)) / len(s1)
        recall = len(s1.intersection(s2)) / len(reljudges[k])
        pre_at_n[k] = round(precision, 3)
        rec_at_n[k] = round(recall, 3)
    return pre_at_n, rec_at_n

In [93]:
def calc_avg_pre(ret_docs, reljudges, cutoff=-1):
    """
    Calculate (mean) average precision for each query in ret_docs
    """
    
    avg_pre, mean_avg_pre = {}, None
    for k, v in ret_docs.items():
        total_rel = 0
        total = 0
        avg_prec = 0
        for i, doc in enumerate(v):
            if doc in reljudges[k] and cutoff == -1:
                total_rel += 1
                total += 1
                precision = total_rel/total
            elif doc in reljudges[k] and i+1 <= cutoff:
                total_rel += 1
                total += 1
                precision = total_rel/total
            else:
                total += 1
                precision = 0
            avg_prec += precision

        avg_pre[k] = round(avg_prec/len(reljudges[k]), 3,)
    
    mean_avg_pre = round(sum(avg_pre.values()) / len(avg_pre), 3)
        
    return avg_pre, mean_avg_pre

In [94]:
import math

def calc_NDCG_at_n(ret_docs, reljudges, n=-1, base=2):
    """
    Calculate NDCG at n for each query in ret_docs
    """
    
    ndcg = {}
    
    for k, v in ret_docs.items():
        
        counts = list(reversed([x for x in range(2,len(reljudges[k])+2)]))
        ideals = {reljudges[k][i]: counts[i] for i in range(len(reljudges[k]))}
        
        add_ons = {}
        if len(v) > len(reljudges[k]):
            for i in range(len(v)-len(reljudges[k])):
                add_ons[i] = 1
        ideals.update(add_ons)
        nums = list(map(ideals.get, v))
        
        systems = {}
        for i, doc in enumerate(v):
            if nums[i] == None:
                systems[doc] = 1
            else:
                systems[doc] = nums[i]
                
        ideal_order = {}
        if n != -1:
            for i, (key, value) in enumerate(ideals.items()):
                if i < n:
                    ideal_order[key] = value
        else:
            ideal_order = ideals
        
        add_ons = {}
        
        
        log = 0
        for i, (doc, rank) in enumerate(ideal_order.items()):
            if i >= len(v):
                break
            elif i < base:
                log += rank
            else:
                log += rank/math.log(i+1, base)

                
        system_order = {}
        if n != -1:
            for i, (key, value) in enumerate(systems.items()):
                if i < n:
                    system_order[key] = value
        else:
            system_order = systems
        
        
        
        system_log = 0
        for i, (doc, rank) in enumerate(system_order.items()):
            if i >= len(v):
                break
            elif i < base:
                system_log += rank
            else:
                system_log += rank/math.log(i+1, base)
        system_log / log
        ndcg[k] = round(system_log / log, 3)
        
    
    return ndcg

In [95]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'lemmatize'))
    ret_docs_index = lsi_model.retrieve_docs(query_name, max_docs=max_docs, normalization='lemmatize')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [96]:
ret_docs_dic

{'home improvement store': ['442299',
  '442291',
  '442210',
  '453998',
  '453991',
  '453930',
  '453910',
  '453920',
  '453310',
  '453220'],
 'diesel fuel supplier': ['447190',
  '447110',
  '336310',
  '445120',
  '454310',
  '454390',
  '441310',
  '221112',
  '454210',
  '424720'],
 'church': ['337127',
  '337121',
  '337124',
  '337122',
  '337125',
  '337214',
  '337110',
  '337211',
  '337910',
  '337920'],
 'farm': ['115115',
  '115210',
  '115310',
  '115113',
  '115116',
  '115111',
  '115112',
  '115114',
  '113110',
  '114210'],
 'seed supplier': ['111191',
  '111199',
  '111998',
  '111992',
  '111991',
  '111219',
  '111120',
  '111419',
  '111160',
  '111150'],
 'accountant': ['541211',
  '541219',
  '541213',
  '541214',
  '541110',
  '541199',
  '541120',
  '541191',
  '541490',
  '541410'],
 'trucking company': ['484121',
  '484230',
  '484122',
  '484220',
  '484110',
  '484210',
  '551114',
  '551111',
  '551112',
  '561110'],
 'exporter': ['522298',
  '522292'

In [97]:
queries_dic

{'home improvement store': ['444110',
  '444120',
  '444130',
  '444190',
  '444210',
  '444220',
  '441110',
  '441120',
  '441210',
  '441222',
  '441228',
  '441310',
  '441320',
  '442110',
  '442210',
  '442291',
  '442299',
  '443141',
  '443142',
  '445110',
  '445120',
  '445210',
  '445220',
  '445230',
  '445291',
  '445292',
  '445299',
  '445310',
  '446110',
  '446120',
  '446130',
  '446191',
  '446199',
  '447110',
  '447190',
  '448110',
  '448120',
  '448130',
  '448140',
  '448150',
  '448190',
  '448210',
  '448310',
  '448320'],
 'diesel fuel supplier': ['424710',
  '424720',
  '424110',
  '424120',
  '424130',
  '424210',
  '424310',
  '424320',
  '424330',
  '424340',
  '424410',
  '424420',
  '424430',
  '424440',
  '424450',
  '424460',
  '424470',
  '424480',
  '424490',
  '424510',
  '424520',
  '424590',
  '424610',
  '424690',
  '424810',
  '424820',
  '424910',
  '424920',
  '424930',
  '424940',
  '424950',
  '424990',
  '423110',
  '423120',
  '423130',
 

In [98]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improvement store': 0.3,
 'diesel fuel supplier': 0.1,
 'church': 0.0,
 'farm': 1.0,
 'seed supplier': 0.0,
 'accountant': 1.0,
 'trucking company': 0.6,
 'exporter': 1.0,
 'grain elevator': 0.0,
 'popcorn store': 0.3,
 'agricultural service': 0.0,
 'warehouse': 0.4,
 'agricultural production': 1.0,
 'ranch': 1.0,
 'holding company': 0.3,
 'farm equipment supplier': 0.0,
 'store': 0.8,
 'grocery store': 0.1,
 'rice mill': 0.9,
 'food product supplier': 0.0,
 'accounting firm': 1.0,
 'produce market': 0.1,
 'pet supply store': 0.9,
 'wholesaler': 1.0,
 'produce wholesaler': 1.0,
 'distribution service': 0.3,
 'crop grower': 1.0,
 'addiction treatment center': 0.6,
 'natural good store': 0.0,
 'orchard': 1.0,
 'lumber store': 0.0,
 'mine': 1.0,
 'transportation service': 1.0,
 'investment company': 0.5,
 'fruit wholesaler': 1.0,
 'real estate agency': 1.0,
 'event venue': 0.0,
 'frozen dessert supplier': 1.0,
 'wine wholesaler importer': 1.0,
 'winery': 0.7,
 'aerospace company': 

In [99]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improvement store': 0.068,
 'diesel fuel supplier': 0.014,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'accountant': 0.204,
 'trucking company': 0.12,
 'exporter': 0.244,
 'grain elevator': 0.0,
 'popcorn store': 0.068,
 'agricultural service': 0.0,
 'warehouse': 0.571,
 'agricultural production': 0.156,
 'ranch': 0.156,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.364,
 'grocery store': 0.023,
 'rice mill': 0.123,
 'food product supplier': 0.0,
 'accounting firm': 0.204,
 'produce market': 0.023,
 'pet supply store': 0.409,
 'wholesaler': 0.141,
 'produce wholesaler': 0.141,
 'distribution service': 0.061,
 'crop grower': 0.156,
 'addiction treatment center': 0.154,
 'natural good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transportation service': 0.2,
 'investment company': 0.122,
 'fruit wholesaler': 0.141,
 'real estate agency': 0.417,
 'event venue': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesaler i

In [100]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improvement store': 0.068,
 'diesel fuel supplier': 0.001,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'accountant': 0.204,
 'trucking company': 0.12,
 'exporter': 0.244,
 'grain elevator': 0.0,
 'popcorn store': 0.016,
 'agricultural service': 0.0,
 'warehouse': 0.44,
 'agricultural production': 0.156,
 'ranch': 0.156,
 'holding company': 1.0,
 'farm equipment supplier': 0.0,
 'store': 0.359,
 'grocery store': 0.003,
 'rice mill': 0.123,
 'food product supplier': 0.0,
 'accounting firm': 0.204,
 'produce market': 0.004,
 'pet supply store': 0.409,
 'wholesaler': 0.141,
 'produce wholesaler': 0.141,
 'distribution service': 0.053,
 'crop grower': 0.156,
 'addiction treatment center': 0.15,
 'natural good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transportation service': 0.2,
 'investment company': 0.052,
 'fruit wholesaler': 0.141,
 'real estate agency': 0.417,
 'event venue': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesaler imp

In [101]:
# lmmatizing performs a bit better in the concept space, perhaps because full words group into concepts al ittle differently?

In [102]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.124

In [103]:
lsi_maps = []
for i in range(1, 11):
    lsi_maps.append(calc_avg_pre(ret_docs_dic, queries_dic, cutoff=i)[1])

In [104]:
lsi_maps

[0.014, 0.029, 0.042, 0.056, 0.068, 0.08, 0.091, 0.102, 0.114, 0.124]

In [105]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improvement store': 0.37,
 'diesel fuel supplier': 0.073,
 'church': 0.021,
 'farm': 0.879,
 'seed supplier': 0.015,
 'accountant': 0.977,
 'trucking company': 0.766,
 'exporter': 0.97,
 'grain elevator': 0.203,
 'popcorn store': 0.23,
 'agricultural service': 0.016,
 'warehouse': 0.759,
 'agricultural production': 0.771,
 'ranch': 0.341,
 'holding company': 1.0,
 'farm equipment supplier': 0.015,
 'store': 0.394,
 'grocery store': 0.084,
 'rice mill': 0.864,
 'food product supplier': 0.015,
 'accounting firm': 0.832,
 'produce market': 0.087,
 'pet supply store': 0.925,
 'wholesaler': 0.791,
 'produce wholesaler': 0.358,
 'distribution service': 0.063,
 'crop grower': 0.889,
 'addiction treatment center': 0.458,
 'natural good store': 0.024,
 'orchard': 0.833,
 'lumber store': 0.015,
 'mine': 0.532,
 'transportation service': 0.114,
 'investment company': 0.361,
 'fruit wholesaler': 0.63,
 'real estate agency': 0.872,
 'event venue': 0.024,
 'frozen dessert supplier': 0.854,
 '

# Stemming

In [106]:
docs = dict(zip(df['naics'], df['stemmed']))

In [107]:
lsi_model = LsiTfidfRecommenderSystem(docs)

In [108]:
ret_docs_dic = {}
queries_dic = {}

for query in relevant_naics.iterrows():
    query_name =  ' '.join(process_query(query[1]['query'], 'stem'))
    ret_docs_index = lsi_model.retrieve_docs(query_name, max_docs=max_docs, normalization='stem')
    ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

    query_docs = query[1]['relevant_naics']
    ret_docs_dic[query_name] = ret_docs
    queries_dic[query_name] = query_docs

In [109]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[0]

{'home improv store': 0.3,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 1.0,
 'seed supplier': 0.0,
 'account': 0.3,
 'truck compani': 0.6,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.3,
 'agricultur servic': 0.0,
 'warehous': 0.4,
 'agricultur product': 1.0,
 'ranch': 1.0,
 'hold compani': 0.3,
 'farm equip supplier': 0.0,
 'store': 0.8,
 'groceri store': 0.1,
 'rice mill': 0.9,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.9,
 'wholesal': 1.0,
 'produc wholesal': 1.0,
 'distribut servic': 0.2,
 'crop grower': 1.0,
 'addict treatment center': 0.6,
 'natur good store': 0.0,
 'orchard': 1.0,
 'lumber store': 0.0,
 'mine': 1.0,
 'transport servic': 1.0,
 'invest compani': 0.7,
 'fruit wholesal': 0.1,
 'real estat agenc': 1.0,
 'event venu': 0.0,
 'frozen dessert supplier': 1.0,
 'wine wholesal import': 0.3,
 'wineri': 0.7,
 'aerospac compani': 0.6,
 'cold storag facil': 0.4,
 'employ agenc': 0.0,
 'plant nurseri':

In [110]:
calc_pre_rec_at_n(ret_docs_dic, queries_dic, n=-1)[1]

{'home improv store': 0.068,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'account': 0.061,
 'truck compani': 0.12,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.068,
 'agricultur servic': 0.0,
 'warehous': 0.571,
 'agricultur product': 0.156,
 'ranch': 0.156,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.364,
 'groceri store': 0.023,
 'rice mill': 0.123,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.409,
 'wholesal': 0.141,
 'produc wholesal': 0.141,
 'distribut servic': 0.041,
 'crop grower': 0.156,
 'addict treatment center': 0.154,
 'natur good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transport servic': 0.2,
 'invest compani': 0.171,
 'fruit wholesal': 0.014,
 'real estat agenc': 0.417,
 'event venu': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesal import': 0.042,
 'wineri': 0.096,
 'aerospac compani': 0.032,
 'cold storag f

In [111]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[0]

{'home improv store': 0.068,
 'diesel fuel supplier': 0.0,
 'church': 0.0,
 'farm': 0.156,
 'seed supplier': 0.0,
 'account': 0.056,
 'truck compani': 0.12,
 'export': 0.0,
 'grain elev': 0.0,
 'popcorn store': 0.016,
 'agricultur servic': 0.0,
 'warehous': 0.571,
 'agricultur product': 0.156,
 'ranch': 0.156,
 'hold compani': 1.0,
 'farm equip supplier': 0.0,
 'store': 0.364,
 'groceri store': 0.003,
 'rice mill': 0.12,
 'food product supplier': 0.0,
 'account firm': 0.0,
 'produc market': 0.0,
 'pet suppli store': 0.409,
 'wholesal': 0.141,
 'produc wholesal': 0.141,
 'distribut servic': 0.034,
 'crop grower': 0.156,
 'addict treatment center': 0.145,
 'natur good store': 0.0,
 'orchard': 0.156,
 'lumber store': 0.0,
 'mine': 0.357,
 'transport servic': 0.2,
 'invest compani': 0.091,
 'fruit wholesal': 0.002,
 'real estat agenc': 0.417,
 'event venu': 0.0,
 'frozen dessert supplier': 0.137,
 'wine wholesal import': 0.024,
 'wineri': 0.096,
 'aerospac compani': 0.016,
 'cold storag fa

In [112]:
calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]

0.117

In [113]:
calc_NDCG_at_n(ret_docs_dic, queries_dic, n=-1, base=2)

{'home improv store': 0.37,
 'diesel fuel supplier': 0.015,
 'church': 0.021,
 'farm': 0.826,
 'seed supplier': 0.015,
 'account': 0.512,
 'truck compani': 0.766,
 'export': 0.026,
 'grain elev': 0.203,
 'popcorn store': 0.221,
 'agricultur servic': 0.016,
 'warehous': 0.834,
 'agricultur product': 0.177,
 'ranch': 0.683,
 'hold compani': 1.0,
 'farm equip supplier': 0.015,
 'store': 0.399,
 'groceri store': 0.077,
 'rice mill': 0.942,
 'food product supplier': 0.015,
 'account firm': 0.021,
 'produc market': 0.024,
 'pet suppli store': 0.939,
 'wholesal': 0.315,
 'produc wholesal': 0.744,
 'distribut servic': 0.052,
 'crop grower': 0.94,
 'addict treatment center': 0.453,
 'natur good store': 0.024,
 'orchard': 0.846,
 'lumber store': 0.015,
 'mine': 0.877,
 'transport servic': 0.113,
 'invest compani': 0.481,
 'fruit wholesal': 0.078,
 'real estat agenc': 0.872,
 'event venu': 0.024,
 'frozen dessert supplier': 0.841,
 'wine wholesal import': 0.33,
 'wineri': 0.826,
 'aerospac compan

In [114]:
df = pd.read_pickle(r'assets/processed_df.pkl')
docs = dict(zip(df['naics'], df['lemmatized']))

In [115]:
concepts = [5, 10, 20, 50, 100, 150, 200]

In [116]:
for concept in concepts:
    lsi_model = LsiTfidfRecommenderSystem(docs, num_concepts=concept)
    ret_docs_dic = {}
    queries_dic = {}

    for query in relevant_naics.iterrows():
        query_name =  ' '.join(process_query(query[1]['query'], 'lemmatize'))
        ret_docs_index = lsi_model.retrieve_docs(query_name, max_docs=max_docs)
        ret_docs = df.iloc[ret_docs_index]['naics'].tolist()

        query_docs = query[1]['relevant_naics']
        ret_docs_dic[query_name] = ret_docs
        queries_dic[query_name] = query_docs
    print(f'Number of concepts: {concept} ' + str(calc_avg_pre(ret_docs_dic, queries_dic, cutoff=-1)[1]))

Number of concepts: 5 0.05
Number of concepts: 10 0.078
Number of concepts: 20 0.115
Number of concepts: 50 0.122
Number of concepts: 100 0.124
Number of concepts: 150 0.12
Number of concepts: 200 0.117
